In [1]:
from functools import reduce
import matplotlib.pyplot as plt
from sklearn import cluster, datasets
import numpy as np
import math

n_samples = 50
blobs,label = datasets.make_blobs(n_samples=n_samples,centers=5, random_state=10,cluster_std=[0.9,1.5,2.0,2.5,3])
data = np.array(blobs)

lbd = 0.1
delta = 1e-4
Lip = 1 + len(data) * lbd / delta
alpha = 1 / Lip
d = 2
n = n_samples
X = np.zeros((n,d))  #initial value

In [2]:
def f(x):
    hg_1 = []
    for i in range(len(x)):
        x_1 = x[i, :] - x
        norm_x = np.linalg.norm(x_1, ord=2, axis=1, keepdims=True)
        hub_mat_1 = norm_x**2 / (2 * delta)
        hub_mat_2 = norm_x - 0.5 * delta
        hub_mat = np.where(norm_x > delta, hub_mat_2, hub_mat_1)
        hg_1.append(hub_mat[i:].sum())
    return lbd * np.array(hg_1).sum() + np.linalg.norm(x-data, axis=1).sum()*0.5

def hub_grad2(x, delta=1e-4):
    hg_1 = np.zeros((n,d))
    for i in range(len(x)):
        x_1 = x[i, :] - x
        norm_x = np.linalg.norm(x_1, ord=2, axis=1, keepdims=True)
        norm_x = np.where(norm_x > delta, norm_x, delta)
        x_1a = x_1 / norm_x
        x_1a = x_1a.sum(axis=0)
        hg_1[i, :] = lbd * x_1a
    return hg_1 + x - data

In [3]:
def agm(x0, tol, maxit=20000, **option):
    """
    Accelerated Gradient method
    """
    xk = x0
    norm = np.zeros(maxit)  #store the norm of gk
    norm[0] = np.linalg.norm(hub_grad2(xk), ord=2)
    y = xk
    alpha = 1 / Lip
    tk_1 = 1
    tk = 1
    print("----Accelerated Gradient Method----")
    print('ITER ;   G.NORM')
    for i in range(1, maxit):
        ng = norm[i - 1]
        if ng <= tol:
            break
        xk, xk_1 = y - alpha * hub_grad2(y), xk
        norm[i] = np.linalg.norm(hub_grad2(xk), ord=2)
        print(i, ng)
        tk, tk_1 = 0.5 * (1 + math.sqrt(1 + 4 * tk_1 ** 2)), tk
        beta = (tk_1 - 1) / tk
        y = xk + beta * (xk - xk_1)
    print("Local Minimizer x =", xk)
    norm = norm[0: i]
    return [xk, norm]

In [4]:
result = agm(X,tol=1e-3)

----Accelerated Gradient Method----
ITER ;   G.NORM
1 53.774326152753645
2 38.719533905430595
3 38.54648514411375
4 38.47357703999191
5 38.44500138607162
6 38.4280291847637
7 38.41909947639121
8 38.41226477794502
9 38.40746710938548
10 38.403148627870344
11 38.399614976002624
12 38.396122023277854
13 38.393008412841425
14 38.38978740143116
15 38.386780622089
16 38.383589874275835
17 38.38049980115168
18 38.37719170087599
19 38.37394485565819
20 38.37046316028778
21 38.36702064061021
22 38.36333393600323
23 38.35967336835408
24 38.35576478194542
25 38.35187614043144
26 38.34773736429857
27 38.34361109956807
28 38.33923002312483
29 38.33485787025414
30 38.330229660709556
31 38.325607993762745
32 38.32072933309542
33 38.31585543366608
34 38.310723993225864
35 38.3055960898291
36 38.30021033274523
37 38.29482725015906
38 38.289186166226486
39 38.28354715012084
40 38.27765010105929
41 38.27175469957989
42 38.2656013273097
43 38.25944931455552
44 38.25303943727455
45 38.24663072202257
46 38.